In [255]:
from astropy import units as u
import numpy as np

# Instrument parameters

### Spectograph design

In [256]:
# telescope focal length
f_tel =  46200 * u.mm
# collimator focal length
f_col =  630 * u.mm
# spectrograph camera focal length
f_cam = 330 * u.mm

### Grating design

In [257]:
# incident grating angle
alpha = np.radians(12.5) * u.radian  # input in degrees
# groove frequency (mm-1)
vg = 900 / u.mm
# groove separation (mm)
d = 1./vg
# spectral order
m = 1.
print('Groove spacing of {:.3} using spectral order {}'.format(d, m))

Groove spacing of 0.00111 mm using spectral order 1.0


### Detector

In [258]:
# size of a CCD pixel
px = 15. * u.micrometer
py = 15. * u.micrometer
# binning of CCD pixels
bx = 2.
by = 2.

# Observational conditions

In [259]:
# nominal slit width (arcseconds)
phi_s = 1.5 * u.arcsec
print('Slid width {:.3}'.format(phi_s))

Slid width 1.5 arcsec


# SALT FWHM
And the PSF in the cross-dispersion direction $ =  \frac{FWHM \cdot M}{S_f} = \frac{FWHM}{S_d}$

SALT is a fixed azimuth mount telescope and the seeing at zenith has to be degraded by the airmass contribution at $37^\circ$ away from zenith    
$FWHM_{z=37} = FWHM_{z=0}\sec^{0.6}(37^\circ)$

Spot size at focal plane assuming a telescope imaging budget of $0.6''$    
$FWHM_{rms} = \sqrt{FWHM^2 + 0.6^2}$

In [260]:
# full width at half maximum @ zenith in Sutherland
fwhm_0 = 1.2 * u.arcsec
# scale FWHM from zenith the fixed azimuth 37 deg away from zenith
z_rad = np.radians(37)  # azimuth offset angle input in degrees
# airmass correction
sec = lambda z: np.arccos(z)
airmass = sec(z_rad)**0.6
fwhm = fwhm_0 * airmass
# assume a 0.6'' imaging error budget
err = 0.6 * u.arcsec
# spot size at focal plane
fwhm = np.sqrt((fwhm*fwhm).value + (err*err).value) * u.arcsec
print('Spot size at focal plane {:.3}'.format(fwhm))

Spot size at focal plane 1.26 arcsec


# Input signals

Assume a continuum object signal of R=21.4 mag, plus sky contribution of R=20.9 mag    
Calculating sky input flux using Vega standards    
$F_* = F_{vega} \times 10^{-0.4m_*}$

In [261]:
def mag2flux(magnitude):
    # convert magnitude to photon flux
    std_flux = 702.0  # erg/s/cm^2/A

    # Star flux relative to Vega
    flux = std_flux * (10.**(-magnitude/2.5))  # ph/s/cm^2/A
    #print('Input flux F_lambda = {:.3} erg/s/cm^2/A'.format(flux))
    
    # Collection area of SALT mirror
    area = lambda radius: np.pi*(radius**2)
    mirror = area(1000/2) - area(400/2)  # cm^2
    print('SALT mirror area = {:.3} cm^2'.format(mirror))
    
    # Incident photons per second on primary
    flux *= mirror # ph/s/A (theoretical)
    return flux

In [262]:
target_flux = mag2flux(21.4)
print('Counts per second on primary = {:.5} erg/s/A'.format(target_flux))

SALT mirror area = 6.6e+05 cm^2
Counts per second on primary = 1.2756 erg/s/A


With the solid angle of the point spread function    
$\Omega = \pi \cdot FWHM^2 / 4$ arcsec$^2$

The diffuse input spectrum is converted from flux per square arcsecond to flux    
$F_\lambda = \Omega F_{\lambda\Omega}$

In [263]:
sky_flux = mag2flux(20.9)
print('Counts per second on primary = {:.3} erg/s/A/arcsec^2'.format(sky_flux))

psf = np.pi * (fwhm / 2.)**2
print('Solid angle of the point spread function {:.3}'.format(psf))

# diffuse input
sky_flux = psf * (sky_flux / u.arcsec**2)  # ph/s/A
print('Sky contribution = {:.5} erg/s/A'.format(sky_flux))

SALT mirror area = 6.6e+05 cm^2
Counts per second on primary = 2.02 erg/s/A/arcsec^2
Solid angle of the point spread function 1.24 arcsec2
Sky contribution = 2.5029 erg/s/A


Scale of slit at telescope focal plane (in mm)    
$S_f = \frac{180 \cdot 3600}{\pi}\frac{1}{f_{tel}}\, [\mbox{''/mm}]$    
$f_{tel}$ the telescope focal length

In [264]:
c = (180.*3600./np.pi)*u.arcsec
Sf = c/f_tel
print('Scale at the focal plane {:.3}'.format(Sf))

Scale at the focal plane 4.46 arcsec / mm


The collimator and camera reimage the incoming light, such that the image of the slit projected onto the CCD is reduced in size by the magnification factor
$M = \frac{f_{cam}}{f_{col}}$,
$f_{col}$ the collimator focal length and
$f_{cam}$ the camera focal length

Given a scale at the detector of
$S_d=S_f/M\, [\mbox{''/mm}]$    

In [265]:
Sd = Sf * (f_col/f_cam)
print('Scale at the detector {:.3}'.format(Sd))

Scale at the detector 8.52 arcsec / mm


In [266]:
phi_mm = phi_s/Sf  # mm
print('Linear width of the slit at the focal plane {:.3}'.format(phi_mm))

Linear width of the slit at the focal plane 0.336 mm


Waveband of a resolution element is determined by the the slit size projected onto the detector in angstroms:    
$\Delta\lambda_{re} = \frac{\phi_s}{S_f} \times \frac{\Lambda\cos(\alpha)}{f_{col}}\, [\mbox{A}]$    
$\phi_s$ the slit width in arcseconds, $\alpha$ the incident grading angle and $\Lambda$ the distance between grooves in angstroms with $1\,mm = 10^7\,A$

In [267]:
# groove period (A)
L = (d/m).to(u.angstrom)  # A for waveband calculations
print('Grating period {:.3}'.format(L))

# step size is constant over wavelength
DLre = phi_mm*(np.cos(alpha)/f_col) * L  # A
print('Width of a resolution element {:.3}'.format(DLre))

# wave range
DL = (900.*u.nm - 360*u.nm).to(u.angstrom)
print('Total wave range {:.3}'.format(DL))

Grating period 1.11e+04 Angstrom
Width of a resolution element 5.79 Angstrom
Total wave range 5.4e+03 Angstrom


# Area of the spectral element

The photons arriving at the detector is dispersed over the geometrical extent of the resolution element in pixel space.

Width of the resolution element = $\frac{dx}{d\lambda} \Delta \lambda_{re}\, [\mbox{mm}]$    
where the linear dispersion $\frac{dx}{d\lambda} = f_{cam}\frac{d\beta}{d\lambda} = \frac{f_{cam}}{\Lambda \cos(\beta)}\, [\mbox{mm/A}]$    
with $\beta = \arcsin(\sin(\alpha) - \frac{\lambda}{\Lambda}) \, [\mbox{rad}]$ the diffraction angle.

In [268]:
# l -- wavelenght [nm]
# a -- grading indicent angle [rad]
# p -- grading period [A]
beta = lambda l, a, p: np.arcsin(l/p.to(u.nm) - np.sin(a))

In [269]:
# grating diffraction angle [rad]
diff_ang = beta(360. * u.nm, alpha, L)  # wavelength input [nm]
# angular dispersion [rad/A]
dbdl = 1. / (L * np.cos(diff_ang))
print('Angular dispersion {:.3}'.format(dbdl))
# linear dispersion [mm/A]
dxdl = f_cam * dbdl
print('Linear dispersion {:.3}'.format(dxdl))
# width of the resolution element
wre = dxdl*DLre
print('Area of resolution element {:.3}'.format(wre))

Angular dispersion 9.05e-05 1 / Angstrom
Linear dispersion 0.0299 mm / Angstrom
Area of resolution element 0.173 mm


The area of the resolution element is the width of the resolution element in the dispersion direction multiplied by the width of the PSF in the cross-dispersion direction    
$A_{re} = \frac{dx}{d\lambda} \Delta \lambda_{re} \times \frac{FWHM}{S_d}\, [\mbox{mm}^2]$ 

In [270]:
Are = wre*fwhm/Sd
print('Projected area of resolution element {:.3}'.format(Are))

Projected area of resolution element 0.0255 mm2


# Area of a binned pixel

Area of a binned pixel $A_b = A_pB_xB_y$ [mm$^2$]    
$A_p$ the area of a CCD pixel, $B_x$ and $B_y$ the $x$ and $y$ binning factors

In [271]:
Ap = (px.to(u.mm))*(py.to(u.mm))  # mm^2
print('Area of a CCD pixel {}'.format(Ap))

Ab = Ap * bx * by
print('Area of a binned pixel {}'.format(Ab))

Area of a CCD pixel 0.000225 mm2
Area of a binned pixel 0.0009 mm2


In [272]:
# binning scale factor
c = Ab / Are
print('Binning scale factor {:.3}'.format(c))

Binning scale factor 0.0354


# SNR calculation

Integrate incoming spectra over resolution element at the focal plane to get incoming count rate, preserving some wavelength dependence at the focal plane

In [273]:
N_lambda = target_flux / (u.angstrom) / (u.second)  # counts/s/A
Ns_lambda = sky_flux / (u.angstrom) / (u.second)  # counts/s/A

Integrated counts    
$N_{re} = \sum\limits_{\Delta\lambda_{re}} N_\lambda d\lambda$ [counts/s/re]

In [274]:
N_target = N_lambda * DLre  # counts/s/re
print('Integrated spectra over resoluton element {:.3} [counts/s/re]'.format(N_target.value))
N_sky = Ns_lambda * DLre  # counts/s/re
print('Sky contribution over resolution element {:.3} [counts/s/re]'.format(N_sky.value))

Integrated spectra over resoluton element 7.38 [counts/s/re]
Sky contribution over resolution element 14.5 [counts/s/re]


Counts per bin    
$N'_{re} = N_{re}\frac{A_b}{A_{re}}$ [counts/s/bin]

In [275]:
N_target *= c  # counts/s/bin
N_sky *= c  # counts/s/bin
print('Counts per bin signal {:.3} [counts/s/bin], sky {:.3} [counts/s/bin]'.format(N_target.value, N_sky.value))
N_total = N_target + N_sky
print('Total signal in counts per second per bin {:.3} [counts/s/bin]'.format(N_total.value))

Counts per bin signal 0.261 [counts/s/bin], sky 0.512 [counts/s/bin]
Total signal in counts per second per bin 0.773 [counts/s/bin]


## Noise contribution

Net exposure time    
$T_e = T - n_r T_r$ [s]    
the time spend counting photons are reduced by the time it takes to read out the CCD

In [276]:
# total time on target
T = 1800 * u.second
# number of readout
nr = 1
# time per readout
ts = 4 * u.second
# readout noise in counts per readout bin
sr = 6

# net exposure time
Te = T - nr*ts
print('Net exposure time {}'.format(Te))

Net exposure time 1796.0 s


Noise per bin comes from three main sources: object, sky and readout

Noise for each signal component    
$\sigma = \sqrt{NT}\ $ [counts], with $N$ number of counts per bin and $T$ exposure time

In [277]:
E_target = np.sqrt(N_target * Te)  # counts
E_sky = np.sqrt(N_sky * Te)  # counts
print('Noise contributions for object {:.3} and sky {:.3} [counts]'.format(E_target, E_sky))

Noise contributions for object 21.6 and sky 30.3 [counts]


Readout noise: $n_r\sigma_r^2$

In [278]:
E_readout = nr * sr**2
print('Readout noise contribution {} [counts^2]'.format(E_readout))

Readout noise contribution 36 [counts^2]


Total noise per bin    
$\sigma^2 = \sigma^2_o + \sigma^2_s + n_r\sigma_r^2 = NT_e + n_r\sigma_r^2$

In [279]:
signal = N_target * Te
noise = np.sqrt(N_total * Te + E_readout)
snr = signal/noise
print('Sensitivity = {:.3}/{:.3} = {:.3}'.format(signal, noise, snr))

Sensitivity = 4.69e+02/37.7 = 12.4
